In [45]:
import execo
import execo_g5k
import execo_engine
import time
import os

# this will be /home/lmascare
HOME_DIR = os.path.expanduser("~")

PATH_TO_SIF_FILE = HOME_DIR + "/bench/docker/images/bench.sif"

SIMULATION_INI = HOME_DIR + "/bench/simulation/setup.ini"
PDI_DEISA_YML = HOME_DIR + "/bench/simulation/io_deisa.yml"
SIM_EXECUTABLE = HOME_DIR + "/bench/simulation/build/main"
DEISA_PATH = HOME_DIR + "/bench/deisa/"

SCHEDULER_FILE = HOME_DIR + "/bench/experiment/scheduler.json"

OUTPUT_DIR = HOME_DIR + "/bench/experiment/output/"


In [46]:
""" 
Get configs from a .ini file.

Arguments:
    config_file (str): Path to the .ini configuration file.
"""
def get_configs(config_file):
    import configparser

    config = configparser.ConfigParser()
    config.read(config_file)

    # Assuming the .ini file has sections and keys
    # Example: [section_name] key_name = value
    configs = {}
    for section in config.sections():
        for key, value in config.items(section):
            configs[key] = value

    return configs

In [47]:
configs = get_configs("simulation/setup.ini")

nb_reserved_nodes = 3

jobs = execo_g5k.oarsub(
    [
        (
            execo_g5k.OarSubmission(f"nodes={nb_reserved_nodes}", walltime=10*60),
            "grenoble",
        )
    ]
)

job_id, site = jobs[0]
print(f"Job {job_id} reserved on site {site}")

nodes = execo_g5k.oar.get_oar_job_nodes(job_id, site)
head_node, nodes = nodes[0], nodes[1:]

print(f"Head node: {head_node}")
print(f"Other nodes: {nodes}")


cores_per_node = execo_g5k.get_host_attributes(head_node)["architecture"]["nb_cores"]
total_simulation_cores = cores_per_node * len(nodes)

print(f"Total simulation cores: {total_simulation_cores}")

Job 2474562 reserved on site grenoble
Head node: Host('dahu-27.grenoble.grid5000.fr')
Other nodes: [Host('dahu-28.grenoble.grid5000.fr'), Host('dahu-4.grenoble.grid5000.fr')]
Total simulation cores: 64


In [48]:
##########################################################
#                   RUNNING SCHEDULER
##########################################################
# Sending the commands for head node

if os.path.exists(SCHEDULER_FILE):
    os.remove(SCHEDULER_FILE)

#redirecting the output to a file
head_node_cmd = (
    f"dask scheduler --scheduler-file {SCHEDULER_FILE}"
)
head_node_process = execo.SshProcess(
    f'singularity exec {PATH_TO_SIF_FILE} bash -c "{head_node_cmd}"',
    head_node,
)
head_node_process.start()

print()
print("#"*20)
print("Output:")
print(head_node_process.stdout)
print("-"*20)
print("Error:")
print(head_node_process.stderr)
print()
print("#"*20)



####################
Output:

--------------------
Error:


####################


In [ ]:

##########################################################
#                   RUNNING SIMULATION
##########################################################

# Starting the simulation 32 - 30 for sim and 2 for dask 


# """
# export OMP_NUM_THREADS=2; \
# export OMP_PROC_BIND=spread; \
# export OMP_PLACES=threads; \
# pdirun /home/lmascare/bench/simulation/build/main /home/lmascare/bench/simulation/setup.ini /home/lmascare/bench/simulation/io_deisa.yml > simulation_output.log 2>&1"

# """
host_list = ",".join([f"{node.address}:{cores_per_node}" for node in nodes])
simulation_cmd = (
    f'pdirun {SIM_EXECUTABLE} {SIMULATION_INI} {PDI_DEISA_YML} > /bench/simulation_output.log 2>&1'
)

# Build the command with two singularity exec calls:
mpi_cmd = (
    # f'singularity exec {PATH_TO_SIF_FILE} bash -c "'
    # 'module load openmpi; '
    'export OMP_NUM_THREADS=2; '
    'export OMP_PROC_BIND=spread; '
    'export OMP_PLACES=threads; '
    f"mpirun -np 4 "#{total_simulation_cores} "
    f"--host {host_list} "
    f"singularity exec {PATH_TO_SIF_FILE} {simulation_cmd}"
)

print("Simulation command:")
print(mpi_cmd)
mpi_process = execo.SshProcess(
    mpi_cmd,
    nodes[0],
)
mpi_process.start()

mpi_process.wait()

print("Output:")
print(mpi_process.stdout)
print("-"*20)
print("Error:")
print(mpi_process.stderr)
print()
print("#"*20)

Simulation command:
export OMP_NUM_THREADS=2; export OMP_PROC_BIND=spread; export OMP_PLACES=threads; mpirun -np 4 --host dahu-28.grenoble.grid5000.fr:32,dahu-4.grenoble.grid5000.fr:32 singularity exec /home/lmascare/bench/docker/images/bench.sif pdirun /home/lmascare/bench/simulation/build/main /home/lmascare/bench/simulation/setup.ini /home/lmascare/bench/simulation/io_deisa.yml > simulation_output.log 2>&1
Output:

--------------------
Error:
Connection to dahu-28.grenoble.grid5000.fr closed.


####################


In [11]:
jobs

[(2474533, 'grenoble')]